# Sora2 Watermark Remover

AI を使って Sora 動画からウォーターマークを除去します。

**新機能:**
- `--frame-step`: フレーム処理間隔（1=全フレーム、2=隔フレーム）
- `--target-fps`: 出力fps指定（0=入力と同じ）

**リポジトリ:** https://github.com/fulfulggg/Sora2WatermarkRemover


In [6]:
# リポジトリのクローンとブランチ切替
!git clone https://github.com/fulfulggg/Sora2WatermarkRemover.git
%cd Sora2WatermarkRemover
!git fetch origin --prune
!git checkout main
!git pull --ff-only
print("✅ リポジトリの準備完了")


Cloning into 'Sora2WatermarkRemover'...
remote: Enumerating objects: 185, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (105/105), done.
remote: Total 185 (delta 83), reused 178 (delta 76), pack-reused 0 (from 0)
Receiving objects: 100% (185/185), 96.69 KiB | 24.17 MiB/s, done.
Resolving deltas: 100% (83/83), done.
/content/Sora2WatermarkRemover
Already on 'main'
Your branch is up to date with 'origin/main'.
Already up to date.
✅ リポジトリの準備完了


## ⚠️ 重要: 依存関係の修正

次のセルを実行後、**必ずランタイムを再起動**してください。


In [7]:
# Pillow 修正（実行後にランタイムを再起動）
!pip uninstall -y pillow PIL
!pip install -U "pillow==10.4.0"
print("\n✅ 完了。ランタイム→ランタイムを再起動 を実行してください。")


Found existing installation: Pillow 9.5.0
Uninstalling Pillow-9.5.0:
  Successfully uninstalled Pillow-9.5.0
  Using cached pillow-10.4.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (9.2 kB)
Using cached pillow-10.4.0-cp312-cp312-manylinux_2_28_x86_64.whl (4.5 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
iopaint 1.2.2 requires Pillow==9.5.0, but you have pillow 10.4.0 which is incompatible.



✅ 完了。ランタイム→ランタイムを再起動 を実行してください。


In [1]:
# 再起動後の確認
import PIL
import PIL._util as u

print(f"Pillow: {PIL.__version__}")
assert PIL.__version__ == "10.4.0" and hasattr(u, "is_directory"), "❌ ランタイムを再起動してください"
print("✅ 依存関係OK")


Pillow: 10.4.0
✅ 依存関係OK


In [2]:
# 競合除去
!pip uninstall -y diffusers huggingface_hub transformers

# ベースライブラリ（ColabのGPU PyTorchを維持するためtorchは入れない）
!pip install -q "iopaint==1.2.2" opencv-python tqdm loguru

# 安定版ピン（Florence trust_remote_codeとiopaintの両立）
!pip install -q "diffusers==0.26.3" "huggingface_hub==0.25.2" "transformers==4.42.4"

!apt-get -qq install -y ffmpeg

# バージョン確認
from importlib.metadata import version as v
print(f"transformers: {v('transformers')}")
print(f"huggingface_hub: {v('huggingface_hub')}")
print(f"diffusers: {v('diffusers')}")
print(f"iopaint: {v('iopaint')}")
print("✅ パッケージインストール完了")


Found existing installation: diffusers 0.26.3
Uninstalling diffusers-0.26.3:
  Successfully uninstalled diffusers-0.26.3
Found existing installation: huggingface-hub 0.25.2
Uninstalling huggingface-hub-0.25.2:
  Successfully uninstalled huggingface-hub-0.25.2
Found existing installation: transformers 4.42.4
Uninstalling transformers-4.42.4:
  Successfully uninstalled transformers-4.42.4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 130.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.6 which is incompatible.
google-adk 1.15.1 requires fastapi<1.0.0,>=0.115.0, but you have fastapi 0.108.0 which is incompatibl

In [3]:
# LaMa モデルの事前ダウンロード
!iopaint download --model lama
print("✅ LaMaモデルのダウンロード完了")


2025-10-09 23:01:05.202 | INFO     | iopaint.runtime:setup_model_dir:82 - Model directory: /root/.cache
/usr/local/lib/python3.12/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is depr

In [4]:
%%bash
set -e
cd /content/Sora2WatermarkRemover
mkdir -p flash_attn
cat > flash_attn/__init__.py <<'PY'
"""
Lightweight stub package for `flash_attn`.
Satisfies import-time checks in Florence-2 (trust_remote_code). Not a real implementation.
"""
def __getattr__(name: str):
    raise RuntimeError(
        f"flash_attn stub in use: attempted to access attribute '{name}'. "
        "This environment runs with eager attention. If you need FlashAttention, "
        "install a matching wheel for your CUDA/SM."
    )
PY
echo "✅ flash_attn stub created at /content/Sora2WatermarkRemover/flash_attn"


✅ flash_attn stub created at /content/Sora2WatermarkRemover/flash_attn


In [5]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ GPU未検出。ランタイム→ランタイムのタイプを変更→GPU")


Device: cuda
GPU: NVIDIA A100-SXM4-40GB


In [6]:
from google.colab import files

print("動画ファイルをアップロード:")
uploaded = files.upload()
input_video = list(uploaded.keys())[0]
print(f"✅ アップロード: {input_video}")


動画ファイルをアップロード:


Saving download.MP4 to download.MP4
✅ アップロード: download.MP4


## パラメータ設定

- `max_bbox_percent`: 検出する最大サイズ（画像比%）
- `frame_step`: 処理間隔（1=全フレーム、2=隔フレーム）
- `target_fps`: 出力fps（0=入力と同じ）


In [7]:
# パラメータ
max_bbox_percent = 10.0
frame_step = 1
target_fps = 0.0
output_video = "output.mp4"

# 処理実行
import os
import time

input_path = f"/content/{input_video}"
output_path = f"/content/{output_video}"

%cd /content/Sora2WatermarkRemover

print(f"入力: {input_path}")
print(f"出力: {output_path}")
print(f"パラメータ: bbox={max_bbox_percent}%, step={frame_step}, fps={target_fps}")
print("\n🚀 処理開始...\n")

t0 = time.time()
!python remwm.py "{input_path}" "{output_path}" \
  --max-bbox-percent {max_bbox_percent} \
  --frame-step {frame_step} \
  --target-fps {target_fps} \
  --force-format=MP4 \
  --overwrite

elapsed = time.time() - t0

if os.path.exists(output_path):
    size_mb = os.path.getsize(output_path) / (1024 * 1024)
    print(f"\n✅ 処理完了")
    print(f"⏱ 処理時間: {elapsed:.1f}秒")
    print(f"📊 ファイルサイズ: {size_mb:.2f} MB")
else:
    print("\n❌ エラー: 出力ファイルが作成されませんでした")


IndentationError: unexpected indent (ipython-input-365528374.py, line 23)

In [ ]:
# 結果のダウンロード
from google.colab import files

files.download(f"/content/{output_video}")
print("✅ ダウンロード完了")


## 使い方のヒント

### 処理を高速化
```python
frame_step = 2  # 隔フレーム処理
```

### 出力fpsを固定
```python
target_fps = 30
```

### 検出感度を調整
```python
max_bbox_percent = 15  # 大きいロゴも検出
```
